In [1]:
import pandas as pd
from datetime import datetime

files = ["job_1999101.json","job_1988758.json", "job_2004889.json","job_2006984.json", "job_2022794.json", "job_2035917.json"]

seen_ids = set()

for file in files:
    file_name = "/dccstor/srosent2/generative/appen/round1_jobs/" + file
    data = pd.read_json(file_name, lines=True)

    for index, row in data.iterrows():
        info = row['results']['judgments'][0]
        # print(row['results'])
            
        seen_ids.add(info['unit_data']['question_id'])

print(len(seen_ids))
# print(seen_ids)

2396


In [2]:
# load tydi json
import glob
import gzip
import json
import spacy

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

In [3]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [6]:
import csv

# up to 19505 to get part2 (2000 each)
# up to 30209 to get part3 (2000 each)
# up to 41213 to get part4 (2000 each)
# up to 53351 to get part5 (2000 each) NOT USED IN APPEN YET

def get_appen_data(data, type, exclude_ids, output_file):
    f = open(output_file, 'w', encoding='UTF8', newline='')
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(["question_id","question","title","url","long_answer","minimal_text"])

    count = 0
    index = 0
    lines = []
    for example in data:
        index += 1
        if index <= 41213:
            continue
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        
        example_id = example['example_id']

        if str(example_id) in exclude_ids:
            continue

        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
            continue
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        sentences = nlp(passage_text)
        
        num_sentences = 0
        for sentence in sentences.sents:
            num_sentences += 1
            passage_sentences += sentence.text + "&nbsp;"

        if num_sentences < 5:
            continue

        count += 1
        if count % 100 == 0:
            print(count)
            print(index)
        if count > 2000:
            break

        writer.writerow([str(example_id), question, example['document_title'], example['document_url'], passage_sentences, minimal_text])
    print(count)
    f.close()
           

In [7]:
get_appen_data(train_data, "la", seen_ids, "/dccstor/srosent2/generative/data_for_appen/full_nq_task_nomin_part5.tsv")

100
41700
200
42235
300
42712
400
43217
500
43746
600
44333
700
44769
800
45342
900
45913
1000
46420
1100
46933
1200
47397
1300
47914
1400
48430
1500
48906
1600
49684
1700
50665
1800
51821
1900
52761
2000
53351
2001
